In [1]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from rembg import remove

import torch
import timm
import cv2
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader 
from torchsampler import ImbalancedDatasetSampler

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

seed = 42
torch.manual_seed(seed)
if device == 'cuda':
  torch.cuda.manual_seed_all(seed)

In [3]:
from efficientnet_pytorch import EfficientNet
pretrained_model = EfficientNet.from_pretrained('efficientnet-b0').to(device)

in_features = pretrained_model._fc.in_features
pretrained_model._fc = nn.Linear(in_features=in_features, out_features=5, bias=True).to(device)

Loaded pretrained weights for efficientnet-b0


In [12]:
shared_layers_list = []
unshared_layers_list = []
end_layer = 7

type = 0 # 0: shared list에 저장 / 1: unshared_list에 저장
for layer in pretrained_model.children():
    if isinstance(layer, nn.ModuleList):                                            # nn.ModuleList에서 층을 나눠야 하므로 별도의 작업이 필요하다.
        shared_modulelist_layers_list = []                                          # ModuleList에 저장된 층들 중 공유되어야 하는 층을 저장한다.
        unshared_modulelist_layers_list = []                                        # ModuleList에 저장된 층들 중 공유되지 않는 층을 저장한다.
        for index, modulelist_layer in enumerate(layer.children()):                 # 각 층을 탐색하면서 알맞은 리스트에 층을 저장한다.
            if index <= end_layer:                                                  # 공유되어야 하는 층이라면 알맞은 리스트에 저장한다.
                shared_modulelist_layers_list.append(modulelist_layer)
            else:                                                                   # 비공유되어야 하는 층이라면 알맞은 리스트에 저장한다.
                unshared_modulelist_layers_list.append(modulelist_layer)
        shared_modulelist_layers = nn.ModuleList(shared_modulelist_layers_list)     # 각 리스트에 저장된 층들을 ModuleList에 저장해 알맞은 list에 append한다.
        shared_layers_list.append(shared_modulelist_layers)
        unshared_modulelist_layers = nn.ModuleList(unshared_modulelist_layers_list)
        unshared_layers_list.append(unshared_modulelist_layers)
        type = 1                                                                    # 이후에 나오는 층들은 unshared list에 저장해야 하므로 type 값을 1로 변경한다.
        continue 
    
    if type == 0: shared_layers_list.append(layer)
    elif type == 1: unshared_layers_list.append(layer)

In [6]:
class EfficientNet_B0(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared_layers = nn.Sequential(
            *shared_layers_list
        )

        self.oil = nn.Sequential(
            *unshared_layers_list
        )
    
        self.sensitive = nn.Sequential(
            *unshared_layers_list
        )

        self.pigmentation = nn.Sequential(
            *unshared_layers_list
        )

        self.wrinkle = nn.Sequential(
            *unshared_layers_list
        )

        self.hydration = nn.Sequential(
            *unshared_layers_list
        )

    def forward(self, input):
        shared_output = self.shared_layers(input)
        oil = self.oil(shared_output)
        sensitive = self.sensitive(shared_output)
        pigmentation = self.pigmentation(shared_output)
        wrinkle = self.wrinkle(shared_output)
        hydration = self.hydration(shared_output)

        return oil, sensitive, pigmentation, wrinkle, hydration

In [7]:
model = EfficientNet_B0()